In [1]:
# install packages
!pip install pytesseract
!pip install pdf2image

You should consider upgrading via the 'C:\Users\lmy\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\lmy\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [2]:
# import packages
import pytesseract
from pdf2image import convert_from_path
import os
import re
from datetime import datetime
import pandas as pd

In [3]:
# Function to extract text from a PDF file
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'  

def extract_text_from_pdf(pdf_path):
    # Convert the PDF to a list of PIL Image objects
    images = convert_from_path(pdf_path)
    text_list = []
    # Loop through each page of the PDF and extract text using Tesseract
    for image in images:
        text = pytesseract.image_to_string(image)
        text_list.append(text)
    # Combine text from all pages into a single string
    text = '\n'.join(text_list)
    return text

In [4]:
date_pattern = r'\b\d{1,2}\s+(?:January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{4}\b'
# Extract year (A) and date (J)
def extract_date(text):
    date_match = re.search(date_pattern, text)
    if date_match:
        date = date_match.group(0)
        date_obj = datetime.strptime(date, '%d %B %Y')
        # change mm/dd/yyyy format
        new_date = date_obj.strftime('%m/%d/%Y') #column J
        year = new_date[-4:] #column A
    else:
        year = date = 'N/A'
        #print("No date found")
    return year, date

In [5]:
def remove_empty(text):
    if text == None:
        return 'N/A'
    text = text.replace('\n', ' ')
    text = text.strip()
    if text == '':
        return 'N/A'
    else:
        return text
    
    
council_pattern1 = r"(?<=session).*?(?=Agenda)"
council_pattern2 = r'([A-Z][A-Za-z]+\s)+(COUNCIL|Council)'

# Extract council/committe name (B)
def extract_council(text):
    match1 = re.search(council_pattern1, text, re.DOTALL)
    match2 = re.search(council_pattern2, text, re.DOTALL)
    council = 'N/A'
    # handle the council after session
    if match1:
        council = match1.group(0).strip()
    # handle the council before session
    if match2:
        council = match2.group(0).strip()
    if 'Economic and Social' in text:
        council = 'Economic and Social Council'
    return remove_empty(council)

In [6]:
session_pattern = r'(?P<session>[\w-]+\ssession)'
# Extract session (C)
def extract_session(text):
    #print(text)
    match = re.search(session_pattern, text)
    if match:
        session = match.group("session")
    else:
        session = 'N/A'
        print("Session information not found.")
    return session

In [7]:
agenda_pattern = r'Agenda item (\d+)(?: \((\w+)\))?(?:\n\n)?'
draft_pattern = r':? *([\w ]*|[\n]*)? ?draft' #': *([\w ]*)?draft'

# Extract Agenda item (D), Agenda detail (E), and countries (F)
def extract_agenda_countries(text):
    match1 = re.search(agenda_pattern, text)
    if match1:
        agenda_item_number = match1.group(1)
        agenda_item_letter = match1.group(2)
        if agenda_item_letter is not None:
            agenda_item = f"{agenda_item_number} ({agenda_item_letter})"
        else:
            agenda_item = agenda_item_number
        
        start_index = match1.end()

        match2 = re.search(draft_pattern, text[start_index:], re.DOTALL)
        if match2:
            end_index = start_index + match2.start()
            content = text[start_index:end_index]
            parts = content.split("\n\n")
            parts = [remove_empty(s) for s in parts]
            agenda_detail = parts[0]
            countries = ''
            for part in parts[1:]:
                if part[-2] == ':':
                    countries += part[:-2]
                else:
                    countries += part
        else:
            print('match2 error')
    
    return agenda_item, remove_empty(agenda_detail), remove_empty(countries)

In [8]:
def split_text(text):
    # delete extracted text
    split_text = re.split(draft_pattern, text, flags=re.IGNORECASE)
    #print(split_text)
    #print(len(split_text))
    part1_text = split_text[0]
    part2_text = split_text[2:]
    part2_text = '\n\n'.join(part2_text)
    return part1_text, part2_text

In [9]:
text_head_pattern = r'(The General Assembly,|The Commission on Human Rights,|The Human Rights Council |The Economic and Social Council,|1\.)'
number_title_pattern = r'^\s*(\d{4})?/?\W*(.*)'

# Extract body title number (G) and body title （H）
def extract_body_title(text):
    title_body_text = re.split(text_head_pattern, text)
    body = ' '.join(title_body_text[1:])
    title_text = title_body_text[0]
    title_text = title_text.replace('\n', ' ')
    
    match1 = re.search(r'[A-Z\d]', title_text)
    if match1:
        index = match1.start()
        title_text = title_text[index-1:]
        match2 = re.match(number_title_pattern, title_text)
        if match2:
            title_number = match2.group(1)
            title = match2.group(2)
    else:
        title_number = 'N/A'
        title = 'N/A'

    if remove_empty(title) == remove_empty(title_text) and len(title) > 400:
        #print(True)
        match3 = re.search(r"[A-Z].*?[A-Z]", title)
        if match3:
            index = match3.end(0) - 1
            body = title[index:]
            body = body.replace("  ", "\n\n")
            title = title[:index]
        #print(title)
    title = remove_empty(title)
    if len(title.split('/')) == 3:
        title = 'N/A'

    return remove_empty(title_number), title, body

In [10]:
footnote_pattern = r'^(\*|\d(?![./])|\d{1}/)\s.*'

# Extract body text (I) and footnote (M)
def extract_body(text):
    # extract the text content from the second non-empty paragraph
    body_text = ''
    footnote = []
    footnote_idx = 1
    paragraphs = text.split('\n\n')
    #print(paragraphs)
    
    for paragraph in paragraphs:
        #for line in paragraph.split('\n'):
            # exclude lines containing specific patterns
        paragraph = paragraph.replace("\n", " ")
        #print(paragraph)
        match = re.search(footnote_pattern, paragraph)
        if match:
            if match.group(0)[0] == '*':
                footnote.append(match.group(0))
            elif match.group(0)[0] == str(footnote_idx):
                footnote.append(match.group(0))
                footnote_idx += 1
        else:
            # if len(paragraph) > 50 or paragraph == "The General Assembly,":
            #     body_text += paragraph.strip() + ' '
            # else:
            if len(paragraph) == 0:
                continue
            if (paragraph[0] == str(footnote_idx) and paragraph[1] == '/') or paragraph[0] == "*":
                footnote_idx += 1
                footnote.append(paragraph) 
            if not re.search(r'(\d{2}-\d{5}|[A-Za-z]/\d{2}/[A-Za-z]\.\d|[A-Za-z ]+Please recycle|Page \d{1})', paragraph):
                body_text += paragraph.strip() + ' '
    #print(footnote)
    footnote = ' '.join(footnote)
    return body_text, footnote

In [11]:
# Function to classify text into column A to N
def classify_text_to_column(text):
    result = []
    
    # Split based on : draft, which is the end of countries
    part1_text, part2_text = split_text(text)
    
    # Get year(A) and date(J)
    year, date = extract_date(part1_text)

    # Get council/committe name (B)
    council = extract_council(part1_text)

    # Get session (C)
    session = extract_session(part1_text)

    # Get agenda item (D), agend detail (E), and countries (F)
    agenda_item, agenda_detail, countries = extract_agenda_countries(text)

    # Get title number (G) and title body (H)
    title_number, title_text, body = extract_body_title(part2_text)

    # Get Body text (I)
    body_text, footnote = extract_body(body)
    
    # Add into result list
    result = [year, council, session, agenda_item, agenda_detail, countries, title_number, title_text, body_text, date, footnote]
    
    return result

In [12]:
# Path to the folder containing the PDF files
pdf_folder = './Dataset'

# Path to the folder where the CSV files will be saved
csv_folder = './csv_folder'

# csv file column list
column_list = ['year', 'Council', 'Session', 'Agenda item', 'Agenda detail', 'cosponsored countries', 
               'body title number',	'body title detail', 'body text', 'date', 'file', 'filecountry', 'footnote', 'scanned']

In [14]:
# Loop through each folder and PDF file and extract text
for country_folder in os.listdir(pdf_folder):
    print('---------------' + country_folder + '---------------' )
    if not os.path.isdir(os.path.join(pdf_folder, country_folder)):
        continue
    
    rows = []
    count = 0
    # Loop through each file 
    #csv_year = 1994
   
    for pdf_file in os.listdir(os.path.join(pdf_folder, country_folder)):
        if not pdf_file.endswith('.pdf'):
            continue
        # check if this file is old-version
        year = int(pdf_file.split('_')[0])
        if year < 1995:
            continue

        pdf_path = os.path.join(pdf_folder, country_folder, pdf_file).replace('\\', '/')
        print(pdf_path)
        # try:
        text = extract_text_from_pdf(pdf_path)
        text_columns = classify_text_to_column(text)
        row = text_columns[:-1] + [pdf_file[:-4], country_folder] + text_columns[-1:] #add column K and L
        
        # check if year was extracted correctlys
        if row[0] != year:
            row[0] = year
        
        # scanned (N), cannot know 
        if int(year) > 1990:
            scanned = 'yes'
        else:
            scanned = 'no'
        row.append(scanned)

        rows.append(row)
        count += 1
        #print(rows)
        if year == 1996:
            df = pd.DataFrame(rows[:-1], columns = column_list)
            csv_path = os.path.join(csv_folder, f"{country_folder}_{year-1}.csv")
            df.to_csv(csv_path, index = False) 
            break
        # except:
        #     print(f"Error extracting text from {pdf_path}")
        #     continue
    break
    # Write the rows to the CSV file
    df = pd.DataFrame(rows, columns = column_list)
    df.to_csv(csv_path, index = False)  

---------------Afghanistan---------------
./Dataset/Afghanistan/1995_171257.pdf
./Dataset/Afghanistan/1995_171258.pdf
./Dataset/Afghanistan/1995_171260.pdf
./Dataset/Afghanistan/1995_171576.pdf
./Dataset/Afghanistan/1995_171580.pdf
./Dataset/Afghanistan/1995_186998.pdf
./Dataset/Afghanistan/1995_199299.pdf
./Dataset/Afghanistan/1995_199514.pdf
./Dataset/Afghanistan/1995_199983.pdf
./Dataset/Afghanistan/1995_200056.pdf
./Dataset/Afghanistan/1995_200283.pdf
./Dataset/Afghanistan/1995_200293.pdf
./Dataset/Afghanistan/1995_200294.pdf
./Dataset/Afghanistan/1995_200295.pdf
./Dataset/Afghanistan/1995_200333.pdf
./Dataset/Afghanistan/1995_200449.pdf
./Dataset/Afghanistan/1995_200507.pdf
./Dataset/Afghanistan/1995_200539.pdf
./Dataset/Afghanistan/1995_200542.pdf
./Dataset/Afghanistan/1995_200749.pdf
./Dataset/Afghanistan/1995_200750.pdf
./Dataset/Afghanistan/1995_200754.pdf
./Dataset/Afghanistan/1995_200898.pdf
./Dataset/Afghanistan/1995_200915.pdf
./Dataset/Afghanistan/1995_201004.pdf
./Datase

UnboundLocalError: local variable 'agenda_item' referenced before assignment

### TEST

In [16]:
test_list = ['1995_201150.pdf'] #, '1995_201197.pdf', '1995_201222.pdf'] 
rows = []
for test_pdf in test_list:
    #if test_pdf == '1994_172588.pdf':
    test_text = extract_text_from_pdf(f'./Dataset/Afghanistan/{test_pdf}')
    print(test_text)
    #break
    p1, p2 = split_text(test_text)
    #print(p1)
    #print(p2)
    # test year and date
    #print(extract_date(p1))
    #print(extract_council(p1))
    #print(extract_session(p1))
    #print(extract_agenda_countries(test_text))
    #t_n, t, b = extract_body_title(p2)
    # print(t_n)
    # print(t)
    #print(b)
    #print(extract_body(b))

        #rt = re.split(title, p2)
        #print(len(rt))
        # year = int(test_pdf.split('_')[0])
        # text_columns = classify_text_to_column(test_text)
        # row = text_columns[:-1] + [test_pdf[:-4], 'country_folder'] + text_columns[-1:] 
    
        # if row[0] != year:
        #     row[0] = year

        # # scanned (N), cannot know 
        # if int(year) > 1990:
        #     scanned = 'yes'
        # else:
        #     scanned = 'no'
        # row.append(scanned)
        # print(row)
        # break
    #rows.append(row)

    #print(rows)
    #df = pd.DataFrame(rows, columns = column_list)

UNITED

NATIONS A

General Assembly
Distr.
LIMITED

A/C.3/50/L. 43
5 December 1995

ORIGINAL: ENGLISH

Fiftieth

session

THIRD COMMITTEE
Agenda item 112 (c)

HUMAN RIGHTS QUESTIONS: HUMAN RIGHTS SITUATIONS AND REPORTS OF

SPECIAL RAPPORTEURS AND REPRESENTATIVES

Afghanistan, Albania, Andorra, Austria, Bangladesh, Bosnia and

Herzegovina, Canada, Chile, Denmark, Djibouti, Egypt, Germany,

Iceland, Ireland, Italy, Jordan, Kuwait, Kyrgyzstan,

Liechtenstein, Luxembourg, Malaysia, Monaco, Morocco,

Netherlands, Pakistan, Portugal, Qatar, Saudi Arabia, Sweden,

Tunisia, Turkey, United Arab Emirates, and United States of
America: draft resolution

Situation of human rights in Kosovo

The General Assembly,

Guided by the Charter of the United Nations, the Universal Declaration of
Human Rights, 1/ the International Covenants on Human Rights, 2/ the
International Convention on the Elimination of All Forms of Racial
Discrimination, 3/ the Convention on the Prevention and Punishment of the
Crime